In [1]:
using LinearAlgebra
using PyPlot
using ForwardDiff
using MatrixCalculus

In [2]:
#Model parameters
g = 9.81 #m/s^2
m = 1.0 #kg 
ℓ = 0.3 #meters

#Thrust and gimbal limits
umin = [0.0; 0.0]
umax = [0.6*m*g; 0.6*m*g]

uhover = [0.5*m*g; 0.5*m*g]

h = 0.025 #time step (50 Hz)

0.025

In [3]:
#Planar Quadrotor Dynamics
function quad_dynamics(x,u)
    J = 0.2*m*ℓ*ℓ
    
    θ = x[3]
    
    ẍ = (1/m)*(u[1] + u[2])*sin(θ)
    ÿ = (1/m)*(u[1] + u[2])*cos(θ) - g
    θ̈ = (1/J)*(ℓ/2)*(u[2] - u[1])
    
    return [x[4:6]; ẍ; ÿ; θ̈]
end

quad_dynamics (generic function with 1 method)

In [4]:
function quad_dynamics_rk4(x,u)
    #RK4 integration with zero-order hold on u
    f1 = quad_dynamics(x, u)
    f2 = quad_dynamics(x + 0.5*h*f1, u)
    f3 = quad_dynamics(x + 0.5*h*f2, u)
    f4 = quad_dynamics(x + h*f3, u)
    return x + (h/6.0)*(f1 + 2*f2 + 2*f3 + f4)
end

quad_dynamics_rk4 (generic function with 1 method)

In [5]:
#First derivatives of dynamics
function dfdx(x,u)
    return ForwardDiff.jacobian(dx->quad_dynamics_rk4(dx,u),x)
end

function dfdu(x,u)
    return ForwardDiff.jacobian(du->quad_dynamics_rk4(x,du),u)
end

dfdu (generic function with 1 method)

In [6]:
#Second derivatives of dynamics
function dAdx(x,u)
    return ForwardDiff.jacobian(dx->vec(dfdx(dx,u)),x)
end

function dBdx(x,u)
    return ForwardDiff.jacobian(dx->vec(dfdu(dx,u)),x)
end

function dAdu(x,u)
    return ForwardDiff.jacobian(du->vec(dfdx(x,du)),u)
end

function dBdu(x,u)
    return ForwardDiff.jacobian(du->vec(dfdu(x,du)),u)
end

dBdu (generic function with 1 method)

In [7]:
Nx = 6     # number of state
Nu = 2     # number of controls
Tfinal = 1.5 # final time
Nt = Int(Tfinal/h)+1    # number of time steps
thist = Array(range(0,h*(Nt-1), step=h));

In [8]:
# Cost weights
Q = Diagonal([ones(3); 0.1*ones(3)]);
R = Array(.01*I(Nu));
Qn = Array(100.0*I(Nx));

In [9]:
#Reference trajectory for flip
x1ref = [LinRange(-3,0,20); zeros(20); LinRange(0,3,21)]
x2ref = [ones(20); LinRange(1,3,10); LinRange(3,1,10); ones(21)]
θref = [zeros(20); LinRange(0,-2*pi,20); -2*pi*ones(21)]
v1ref = [6.0*ones(20); zeros(20); 6.0*ones(21)]
v2ref = [zeros(20); 8.0*ones(10); -8.0*ones(10); zeros(21)]
ωref = [zeros(20); -4*pi*ones(20); zeros(21)]
xref = [x1ref'; x2ref'; θref'; v1ref'; v2ref'; ωref']

6×61 Array{Float64,2}:
 -3.0  -2.84211  -2.68421  -2.52632  …   2.7       2.85      3.0
  1.0   1.0       1.0       1.0          1.0       1.0       1.0
  0.0   0.0       0.0       0.0         -6.28319  -6.28319  -6.28319
  6.0   6.0       6.0       6.0          6.0       6.0       6.0
  0.0   0.0       0.0       0.0          0.0       0.0       0.0
  0.0   0.0       0.0       0.0      …   0.0       0.0       0.0

In [10]:
#Reference trajectory for straight line
x1ref = Array(LinRange(-2,2,101))
x2ref = Array(ones(101))
θref = Array(zeros(101))
v1ref = Array((4.0/3.0)*ones(101))
v2ref = Array(zeros(101))
ωref = Array(zeros(101))
xref = [x1ref'; x2ref'; θref'; v1ref'; v2ref'; ωref']

6×101 Array{Float64,2}:
 -2.0      -1.96     -1.92     …  1.88     1.92     1.96     2.0
  1.0       1.0       1.0         1.0      1.0      1.0      1.0
  0.0       0.0       0.0         0.0      0.0      0.0      0.0
  1.33333   1.33333   1.33333     1.33333  1.33333  1.33333  1.33333
  0.0       0.0       0.0         0.0      0.0      0.0      0.0
  0.0       0.0       0.0      …  0.0      0.0      0.0      0.0

In [11]:
function stage_cost(x,u,k)
    return 0.5*(x-xref[:,k])'*Q*(x-xref[:,k]) + 0.5*(u-uhover)'*R*(u-uhover)
    #return 0.5*(x-xgoal)'*Q*(x-xgoal) + 0.5*(u-uhover)'*R*(u-uhover)
end

stage_cost (generic function with 1 method)

In [12]:
function terminal_cost(x)
    return 0.5*(x-xref[:,Nt])'*Qn*(x-xref[:,Nt])
    #return 0.5*(x-xgoal)'*Qn*(x-xgoal)
end

terminal_cost (generic function with 1 method)

In [13]:
function cost(xtraj,utraj)
    J = 0
    for k = 1:(Nt-1)
        J += stage_cost(xtraj[:,k],utraj[:,k],k)
    end
    J += terminal_cost(xtraj[:,Nt])
    return J
end

cost (generic function with 1 method)

In [14]:
function V(x,k)
    Δx = x-xtraj[:,k]
    return 0.5*Δx'*P[:,:,k]*Δx + p[:,k]'*Δx
end

V (generic function with 1 method)

In [15]:
function S(x,u,k)
    return stage_cost(x,u,k) + V(quad_dynamics_rk4(x,u),k+1)
end

S (generic function with 1 method)

In [22]:
#Initial guess trajectory (hover)
x0 = [-2.0; 1.0; 0; 0; 0; 0]
xgoal = [2.0; 1.0; 0; 0; 0; 0]
xtraj = kron(ones(1,Nt), x0)
utraj = kron(ones(1,Nt-1), uhover)
J = cost(xtraj,utraj)

438.3902222222222

In [23]:
#DDP Algorithm
p = zeros(Nx,Nt)
P = zeros(Nx,Nx,Nt)
j = ones(Nu,Nt-1)
K = zeros(Nu,Nx,Nt-1)
ΔJ = 0.0

Gxx = zeros(Nx,Nx)
Guu = zeros(Nu,Nu)
Gxu = zeros(Nx,Nu)
Gux = zeros(Nu,Nx)

iter = 0
tol = 1.0
β = 1e-6
while tol > 1e-5
    iter += 1
    
    p = zeros(Nx,Nt)
    P = zeros(Nx,Nx,Nt)
    j = zeros(Nu,Nt-1)
    K = zeros(Nu,Nx,Nt-1)
    ΔJ = 0.0

    p[:,Nt] = ForwardDiff.gradient(terminal_cost,xtraj[:,Nt])
    P[:,:,Nt] = ForwardDiff.hessian(terminal_cost,xtraj[:,Nt])
    
    #Backward Pass
    failed = false
    for k = (Nt-1):-1:1
        #Calculate derivatives
        q = ForwardDiff.gradient(dx->stage_cost(dx,utraj[:,k],k),xtraj[:,k])
        Q = ForwardDiff.hessian(dx->stage_cost(dx,utraj[:,k],k),xtraj[:,k])
    
        r = ForwardDiff.gradient(du->stage_cost(xtraj[:,k],du,k),utraj[:,k])
        R = ForwardDiff.hessian(du->stage_cost(xtraj[:,k],du,k),utraj[:,k])
    
        A = dfdx(xtraj[:,k],utraj[:,k])
        B = dfdu(xtraj[:,k],utraj[:,k])
    
        gx = q + A'*p[:,k+1]
        gu = r + B'*p[:,k+1]
    
        Gxx = Q + A'*P[:,:,k+1]*A
        Guu = R + B'*P[:,:,k+1]*B
        Gux = B'*P[:,:,k+1]*A
        
        if tol < 1e-2
            #Add full Newton terms
            Ax = dAdx(xtraj[:,k],utraj[:,k])
            Bx = dBdx(xtraj[:,k],utraj[:,k])
            Bu = dBdu(xtraj[:,k],utraj[:,k])
            
            Gxx += kron(p[:,k+1]',I(Nx))*comm(Nx,Nx)*Ax
            Guu += kron(p[:,k+1]',I(Nu))*comm(Nx,Nu)*Bu
            Gux += kron(p[:,k+1]',I(Nu))*comm(Nx,Nu)*Bx
        end
    
        #Regularization
        Gxx_reg = Gxx + A'*β*I*A
        Guu_reg = Guu + B'*β*I*B
        Gux_reg = Gux + B'*β*I*A
        C = cholesky(Symmetric([Gxx_reg Gux_reg'; Gux_reg Guu_reg]), check=false)
        if !issuccess(C)
            β = 2*β
            failed = true
            display(β)
            break
        end
    
        j[:,k] .= Guu_reg\gu
        K[:,:,k] .= Guu_reg\Gux_reg
    
        p[:,k] .= gx - K[:,:,k]'*gu + K[:,:,k]'*Guu*j[:,k] - Gux'*j[:,k]
        P[:,:,k] .= Gxx + K[:,:,k]'*Guu*K[:,:,k] - Gux'*K[:,:,k] - K[:,:,k]'*Gux
    
        ΔJ += gu'*j[:,k]
    end
    #display(j)

    #Forward rollout with line search
    if !failed
    xn = zeros(Nx,Nt)
    un = zeros(Nu,Nt-1)
    xn[:,1] = xtraj[:,1]
    α = 1.0

    for k = 1:(Nt-1)
        un[:,k] .= utraj[:,k] - α*j[:,k] - K[:,:,k]*(xn[:,k]-xtraj[:,k])
        xn[:,k+1] .= quad_dynamics_rk4(xn[:,k],un[:,k])
    end
    Jn = cost(xn,un)
    
    while Jn > (J - 1e-2*α*ΔJ)
        α = 0.5*α
        for k = 1:(Nt-1)
            un[:,k] .= utraj[:,k] - α*j[:,k] - K[:,:,k]*(xn[:,k]-xtraj[:,k])
            xn[:,k+1] .= quad_dynamics_rk4(xn[:,k],un[:,k])
        end
        Jn = cost(xn,un)
    end
    #display(α)
    
    J = Jn
    xtraj .= xn
    utraj .= un
    
    tol = maximum(abs.(j[:]))
    β = max(0.9*β, 1e-6)
    end
end

2.0e-6

4.0e-6

8.0e-6

1.6e-5

3.2e-5

...

0.262144

0.524288

0.9437184

0.9027551703859202

0.8635699989091149

In [24]:
iter

46

In [ ]:
#Set up visualization
using MeshCat
using RobotZoo: Quadrotor, PlanarQuadrotor
using CoordinateTransformations, Rotations, Colors, StaticArrays, RobotDynamics

function set_mesh!(vis, model::L;
        scaling=1.0, color=colorant"black"
    ) where {L <: Union{Quadrotor, PlanarQuadrotor}} 
    # urdf_folder = joinpath(@__DIR__, "..", "data", "meshes")
    urdf_folder = @__DIR__
    # if scaling != 1.0
    #     quad_scaling = 0.085 * scaling
    obj = joinpath(urdf_folder, "quadrotor_scaled.obj")
    if scaling != 1.0
        error("Scaling not implemented after switching to MeshCat 0.12")
    end
    robot_obj = MeshFileGeometry(obj)
    mat = MeshPhongMaterial(color=color)
    setobject!(vis["robot"]["geom"], robot_obj, mat)
    if hasfield(L, :ned)
        model.ned && settransform!(vis["robot"]["geom"], LinearMap(RotX(pi)))
    end
end

function visualize!(vis, model::PlanarQuadrotor, x::StaticVector)
    py,pz = x[1], x[2]
    θ = x[3]
    settransform!(vis["robot"], compose(Translation(0,py,pz), LinearMap(RotX(-θ))))
end

function visualize!(vis, model, tf::Real, X)
    fps = Int(round((length(X)-1)/tf))
    anim = MeshCat.Animation(fps)
    n = state_dim(model)
    for (k,x) in enumerate(X)
        atframe(anim, k) do
            x = X[k]
            visualize!(vis, model, SVector{n}(x)) 
        end
    end
    setanimation!(vis, anim)
end

In [ ]:
vis = Visualizer()
model = PlanarQuadrotor()
set_mesh!(vis, model)
render(vis)

In [ ]:
X1 = [SVector{6}(x) for x in eachcol(xtraj)];
visualize!(vis, model, thist[end], X1)